In [1]:
import intake
import xarray as xr
import zarr
import numpy as np

import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../src/')))
#sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../weather_routing_old/src/')))

from weather_router import isochronal_weather_router, polar, point_validity, visualize


In [2]:
#LA = (33.6, -124)
#Honolulu = (21.3, -157.8)

Cape = (-34, 18)
Rio = (-22.9, -43.2)

min_lat = min(Cape[0], Rio[0]) - 5  
max_lat = max(Cape[0], Rio[0]) + 5  

min_lon = min(Cape[1], Rio[1]) - 1  
max_lon = max(Cape[1], Rio[1]) + 1  

In [3]:

if not os.path.exists('cache.zarr/'):
    print('Downloading data...')
    ds = xr.open_zarr(
        'gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3',
        chunks=None,
        storage_options=dict(token='anon'),
    )
    ds = ds.rename({'latitude':'lat', 'longitude':'lon'})
    ds = ds[['10m_u_component_of_wind', '10m_v_component_of_wind']]
    ds.coords['lon'] = ((ds.coords['lon'] + 180) % 360) - 180
    ds = ds.sortby(ds.lon)
    ds = ds.sel(lat=slice(max_lat, min_lat), lon=slice(min_lon, max_lon))
    ds = ds.sel(time = slice('2025-01-01T12:00:00', '2025-01-18T12:00:00'))
    ds = ds.load()
    u10 = ds['10m_u_component_of_wind']
    v10 = ds['10m_v_component_of_wind']
    tws = xr.ufuncs.hypot(v10, u10)
    tws = tws*1.94384 #convert m/s to knots
    twd = (180 + np.rad2deg(np.arctan2(u10, v10))) % 360
    ds = tws.to_dataset(name = 'tws')
    ds['twd'] = twd
    ds.to_zarr('cache.zarr')
else:
    ds = xr.open_zarr('cache.zarr')

In [4]:
#ds = ds.isel(time = slice(72,-1))

In [5]:
def get_wind(t, lat, lon):
    tws_sel = ds.tws.sel(time = t, method = 'nearest')
    tws_sel = tws_sel.sel(lat = lat, lon = lon, method = 'nearest')
    twd_sel = ds.twd.sel(time = t, method = 'nearest')
    twd_sel = twd_sel.sel(lat = lat, lon = lon, method = 'nearest')
    return (np.float32(twd_sel.values), np.float32(tws_sel.values))

In [6]:
volvo70_polar = polar.Polar('../test/volvo70.pol')

point_valid = point_validity.land_sea_mask().point_validity_arr

weatherrouter = isochronal_weather_router.weather_router(volvo70_polar, 
                                                         get_wind, 
                                                         time_steps = ds.time.values,
                                                         step = 1,
                                                         start_point = Cape,
                                                         end_point = Rio,
                                                         point_validity_extent = [min_lat, min_lon, max_lat, max_lon],
                                                         spread = 130,
                                                         wake_lim = 30,
                                                         rounding = 2,
                                                         n_points=30,
                                                         )

In [7]:
weatherrouter.route()

step 1 number of isochrone points 13 dist to finish 3240.9
step 2 number of isochrone points 18 dist to finish 3228.8
step 3 number of isochrone points 20 dist to finish 3216.2
step 4 number of isochrone points 22 dist to finish 3203.6
step 5 number of isochrone points 23 dist to finish 3188.2
step 6 number of isochrone points 23 dist to finish 3173.0
step 7 number of isochrone points 22 dist to finish 3156.5
step 8 number of isochrone points 25 dist to finish 3139.3
step 9 number of isochrone points 27 dist to finish 3123.2
step 10 number of isochrone points 23 dist to finish 3107.3
step 11 number of isochrone points 27 dist to finish 3091.5
step 12 number of isochrone points 25 dist to finish 3073.5
step 13 number of isochrone points 26 dist to finish 3052.8
step 14 number of isochrone points 25 dist to finish 3031.8
step 15 number of isochrone points 22 dist to finish 3013.7
step 16 number of isochrone points 21 dist to finish 2997.2
step 17 number of isochrone points 22 dist to fin

In [8]:
route_df = weatherrouter.get_fastest_route()
route_df

,lat,lon,time,twd,tws,pos,next_pos,heading,twa,base_boat_speed,is_tacking,boat_speed,hours_elapsed,days_elapsed
0,-34.000000,18.000000,2025-01-01 12:00:00,232.094025,10.638941,"(-34.0, 18.0)","(-33.91442701217998, 17.768982280463707)",294.000000,61.905975,12.60,False,12.60,0,0.000000
1,-33.914427,17.768982,2025-01-01 13:00:00,235.547287,12.729176,"(-33.91442701217998, 17.768982280463707)","(-33.772228858767036, 17.51583059842409)",304.000000,68.452713,15.24,False,15.24,1,0.041667
2,-33.772229,17.515831,2025-01-01 14:00:00,229.544617,13.184561,"(-33.772228858767036, 17.51583059842409)","(-33.53733394471476, 17.311232239298327)",324.000000,94.455383,17.42,False,17.42,2,0.083333
3,-33.537334,17.311232,2025-01-01 15:00:00,221.543182,13.329449,"(-33.53733394471476, 17.311232239298327)","(-33.38560432044475, 17.04234458774852)",304.000000,82.456818,16.26,False,16.26,3,0.125000
4,-33.385604,17.042345,2025-01-01 16:00:00,219.955811,14.590051,"(-33.38560432044475, 17.04234458774852)","(-33.320751374990536, 16.732904045323465)",284.000000,64.044189,16.00,False,16.00,4,0.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,-23.325902,-43.093364,2025-01-13 00:00:00,141.784500,5.056849,"(-23.32590203422517, -43.093363636789505)","(-23.277346148175894, -43.05353609330827)",37.000000,-104.784500,7.30,True,3.65,276,11.500000
277,-23.277346,-43.053536,2025-01-13 01:00:00,139.377045,4.103590,"(-23.277346148175894, -43.05353609330827)","(-23.20179907715793, -42.984580282288704)",40.000000,-99.377045,5.92,False,5.92,277,11.541667
278,-23.201799,-42.984580,2025-01-13 02:00:00,139.020447,2.868200,"(-23.20179907715793, -42.984580282288704)","(-23.199132998842014, -43.01215295701329)",276.000000,136.979553,3.06,True,1.53,278,11.583333
279,-23.199133,-43.012153,2025-01-13 03:00:00,157.068924,2.066015,"(-23.199132998842014, -43.01215295701329)","(-23.192143042544103, -43.06011582445876)",279.000000,121.931076,2.68,False,2.68,279,11.625000


In [9]:
isochrones = weatherrouter.get_isochrones_latlon()

In [10]:
viz = visualize.visualize(ds.sel(time = slice(route_df.time.values[0], route_df.time.values[-1])), Cape, Rio, route_df, isochrone=isochrones, filename='route1')
viz.return_plot()

BokehModel(combine_events=True, render_bundle={'docs_json': {'adbebb14-c7e4-4069-9e9d-bf0471b142c3': {'version…

In [11]:
initial_route = weatherrouter.get_fastest_route(stats=False)
initial_isochrones = weatherrouter.get_isochrones()

optimized_route = weatherrouter.optimize(initial_route, initial_isochrones)

step 1 number of isochrone points 13 dist to finish 3240.9
step 2 number of isochrone points 40 dist to finish 3228.8
step 3 number of isochrone points 57 dist to finish 3216.2
step 4 number of isochrone points 26 dist to finish 3203.5
step 5 number of isochrone points 59 dist to finish 3188.2
step 6 number of isochrone points 31 dist to finish 3172.8
step 7 number of isochrone points 32 dist to finish 3156.1
step 8 number of isochrone points 50 dist to finish 3138.8
step 9 number of isochrone points 49 dist to finish 3120.8
step 10 number of isochrone points 59 dist to finish 3104.1
step 11 number of isochrone points 34 dist to finish 3088.4
step 12 number of isochrone points 54 dist to finish 3072.7
step 13 number of isochrone points 29 dist to finish 3054.4
step 14 number of isochrone points 44 dist to finish 3033.4
step 15 number of isochrone points 41 dist to finish 3014.1
step 16 number of isochrone points 47 dist to finish 2995.0
step 17 number of isochrone points 52 dist to fin

In [12]:
route_df = weatherrouter.get_fastest_route(use_optimized = True)
route_df

,lat,lon,time,twd,tws,pos,next_pos,heading,twa,base_boat_speed,is_tacking,boat_speed,hours_elapsed,days_elapsed
0,-34.000000,18.000000,2025-01-01 12:00:00,232.094025,10.638941,"(-34.0, 18.0)","(-33.800723979550014, 17.825876008314737)",324.00000,91.905975,14.78,False,14.78,0,0.000000
1,-33.800724,17.825876,2025-01-01 13:00:00,225.420486,12.146090,"(-33.800723979550014, 17.825876008314737)","(-33.70316572542025, 17.5632384183725)",294.00000,68.579514,14.36,False,14.36,1,0.041667
2,-33.703166,17.563238,2025-01-01 14:00:00,229.544617,13.184561,"(-33.70316572542025, 17.5632384183725)","(-33.55648268704515, 17.302767510682006)",304.00000,74.455383,15.72,False,15.72,2,0.083333
3,-33.556483,17.302768,2025-01-01 15:00:00,221.543182,13.329449,"(-33.55648268704515, 17.302767510682006)","(-33.452930749412054, 17.024842083218115)",294.00000,72.456818,15.24,False,15.24,3,0.125000
4,-33.452931,17.024842,2025-01-01 16:00:00,219.955811,14.590051,"(-33.452930749412054, 17.024842083218115)","(-33.27651265027081, 16.712688522488047)",304.00000,84.044189,18.90,False,18.90,4,0.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,-23.321323,-42.604720,2025-01-11 18:00:00,175.966263,9.491061,"(-23.321322955254978, -42.6047200461372)","(-23.430370633746602, -42.69429882713071)",217.00000,41.033737,8.20,False,8.20,246,10.250000
247,-23.430371,-42.694299,2025-01-11 19:00:00,163.784042,7.838160,"(-23.430370633746602, -42.69429882713071)","(-23.34444900832425, -42.77853857744019)",318.00000,154.215958,6.94,False,6.94,247,10.291667
248,-23.344449,-42.778539,2025-01-11 20:00:00,152.387955,10.498222,"(-23.34444900832425, -42.77853857744019)","(-23.249029916522396, -42.9113851512398)",308.00000,155.612045,9.30,False,9.30,248,10.333333
249,-23.249030,-42.911385,2025-01-11 21:00:00,141.190765,9.998202,"(-23.249029916522396, -42.9113851512398)","(-23.16764344299585, -43.00966559328329)",312.00000,170.809235,7.30,False,7.30,249,10.375000


In [12]:
restricted_isochrones = weatherrouter.get_optimized_isochrones_latlon()

In [13]:
viz = visualize.visualize(ds.sel(time = slice(route_df.time.values[0], route_df.time.values[-1])), Cape, Rio, route_df, isochrone=restricted_isochrones, filename='route1')
viz.return_plot()

BokehModel(combine_events=True, render_bundle={'docs_json': {'0000f403-a85f-468a-80d5-dbd7b766da2a': {'version…